# `bavapi` Jupyter notebook demo

In this notebook we will demo the functionality of `bavapi`, a python package to access BAV data from the WPPBAV [Fount](https:/fount.wppbav.com).

To learn more about `bavapi` and how to use it, please visit the [documentation](https://wppbav.github.io/bavapi-sdk-python/).

To learn more about the `BAV API` and how to use it, please visit the [API documentation](https://developer.wppbav.com).

## Motivation

The reason for using `bavapi` is that, while the Fount API provides a way to programmatically access BAV data, it is primarily intended for integrating BAV data into web applications.

This means that the way the data is provided makes it a bit harder to use in data science analysis:

- The data is delivered in JSON format, which has to be parsed into a DataFrame.
- Some of the data is composed of nested dictionaries, which makes it a bit harder to parse.
- The data is delivered in a paginated form, meaning that it is chunked into batches of 25 rows by default. A typical BSS would take about 35 requests to get all data at 25 rows per request.

`bavapi` provides a simple solution to get this data in a workable format for data science workflows.

## Fount API Token

A Fount API bearer token (aka key) is required to use `bavapi`, which you can retrieve from your [Fount](https:/fount.wppbav.com) account settings under the "API" tab.

In [ ]:
TOKEN = "your_token"  # paste your token here, or follow the instructions below

The following cells uses `python-dotenv` ([learn more](https://github.com/theskumar/python-dotenv)) to load the token from a `.env` file, as recommended in the `bavapi` [documentation](https://wppbav.github.io/bavapi-sdk-python/latest/getting-started/authentication/).

>__**💡 Tip:**__ Uncomment the cell below to install `python-dotenv`

In [ ]:
# %pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
TOKEN = os.environ["FOUNT_API_KEY"]

## Using `bavapi` to get BAV data

To get started, install `bavapi` by running the following command:

```
pip install wpp-bavapi
```

For more information, see the [installation](https://wppbav.github.io/bavapi-sdk-python/latest/getting-started/installation/) instructions in the `bavapi` documentation.

>__**💡 Tip:**__ Uncomment the cell below to install `bavapi`

In [ ]:
# %pip install wpp-bavapi

Once you have installed `bavapi`, import it and start using one of the available endpoint functions:

- `audiences`
- `brand_metric_groups`
- `brand_metrics`
- `brands`
- `brandscape_data`
- `categories`
- `collections`
- `sectors`
- `studies`

In [ ]:
import bavapi

### Brands endpoint

Now we can query the database. For example, we can request which brands exist with `"Swatch"` in their name.

In [ ]:
swatch = bavapi.brands(TOKEN, "Swatch")

And get a DataFrame with all available information for brands that contain "Swatch" in their name.

In [ ]:
swatch

### Studies endpoint

Similarly, you can query the `studies` endpoint to get a summary of all BAV studies, with advanced filtering capabilities.

In [ ]:
uk_studies = bavapi.studies(TOKEN, "GB", 2022)
uk_studies

### Audiences endpoint

In [ ]:
audiences = bavapi.audiences(TOKEN, active=1)
audiences.head()

### Brandscape data endpoint
#### Valid queries

The `brandscape-data` endpoint has been restricted to require at least one of these specific set of filters:

- Study + Audience + Brand + Category
- Country + Year + Audience
- Brand + Audience + Country + Year

You should read these from left to right. A combination of "Study + Audience" works just as well as "Study + Audience + Brand". However, "Category + Audience" will not.

If you use Country or Year filters, you must use both filters together.

If a query does not have any of these combinations of filters, it will raise a `ValidationError`

In [ ]:
uk_bss = bavapi.brandscape_data(TOKEN, audiences=1)  # raises ValidationError

In [ ]:
uk_bss = bavapi.brandscape_data(TOKEN, country_code="GB", year_number=2020, audiences=1)
uk_bss.head()

### Downloading and using Reference Classes

The `brandscape_data` endpoint will always return all audiences (aka bases) for a study unless audiences are specified. This means queries without base information can require substantially more requests (~35 audiences, a typical study with all audiences would need over 500 total requests).

To more easily use this endpoint, you can download the reference classes for Audiences and Countries, which will make filtering more explicit.

For example, normally you would have to specify the audience by using its Fount ID. This can be retrieved from the `audiences` endpoint, and used like so:

```py
bavapi.brandscape_data(TOKEN, country_code="GB", audiences=123)
```

Or alternatively, use the reference class:

```py
bavapi.brandscape_data(TOKEN, country_code="GB", audiences=Audiences.ALL_ADULTS)
```

To download these references, you must specify your Fount API key in a `.env` file at the top of your current working directory.

Run the following to generate reference files in a new directory `bavapi_refs` (replacing `"TOKEN"` with your token):

```prompt
bavapi-gen-refs --all -t "TOKEN"
```

>__**⚠ WARNING:**__ DO NOT PUSH REFERENCE FILES TO GIT! Add `bavapi_refs/` to your `.gitignore` file.

>__**💡 Tip:**__ Uncomment the cell below to download reference files. Replace `TOKEN` with your API key. If you have set up a `.env` file with `FOUNT_API_KEY` as an environment variable, you can use the following cell instead.

In [ ]:
# !bavapi-gen-refs --all -t "TOKEN"

In [ ]:
# !bavapi-gen-refs --all

Now you should be ready to import reference classes.

In [ ]:
from bavapi_refs import Audiences

The same can be done with countries. Use:

```py
from bavapi_refs import Countries
```

## More examples

Here are some additional examples of more complex queries:

In [ ]:
# Get one single brand by ID
swatch = bavapi.brands(
    TOKEN, brand_id=2985  # Note that this ID is the Fount ID, not the one in `bav_key`
)
swatch

In [ ]:
# Get US studies that ended fielding after `2021-09-01`
us_studies = bavapi.studies(
    TOKEN, "US", filters={"data_updated_since": "2021-09-01 00:00:00"}
)
us_studies.head()

In [ ]:
# Get Spanish data for 2022 for the `Adults with kids` base sorted by Differentiation in descending order (highest first)
es_bss = bavapi.brandscape_data(
    TOKEN,
    "ES",
    2022,
    [Audiences.ADULTS_WITH_KIDS, Audiences.ADULTS_WITH_KIDS_0_11],
    sort="-differentiation_rank",
)
es_bss.head()

In [ ]:
# Get Spanish data for 2020 and 2022
es_bss = bavapi.brandscape_data(TOKEN, "ES", [2019, 2022], Audiences.ALL_ADULTS)
es_bss.head()

In [ ]:
# Get French and Spanish data for 2022
es_fr_bss = bavapi.brandscape_data(TOKEN, ["ES", "FR"], 2022, Audiences.ALL_ADULTS)
es_fr_bss.head()

### Using `bavapi.Query` with endpoint functions

>__New in `v0.11.0`__.

Instead of passing parameters to the functions, you can also create a `bavapi.Query` object and use it instead.

One benefit of doing this is that you could reuse the `Query`, or even save as a file and load it for reproducibility.

Let's create a `Query` object:

In [ ]:
query = bavapi.Query(
    filters=bavapi.filters.StudiesFilters(
        country_codes=["US", "GB"], year_numbers=2023
    ),
    include="country",
)

You can use it in the `studies` function:

In [ ]:
anglo_studies = bavapi.studies(TOKEN, query=query)
anglo_studies.head()

You can save it as a JSON file:

In [ ]:
import json

with open("query.json", "w", encoding="utf-8") as file:
    json.dump(query.model_dump(), file)

And load again for later reuse:

In [ ]:
with open("query.json", encoding="utf-8") as file:
    query = bavapi.Query(**json.load(file))

loaded_anglo_studies = bavapi.studies(TOKEN, query=query)
loaded_anglo_studies.head()

>__**Note:**__ The loaded filter object will be of type `FountFilters` instead of the original type, but the requests should work as intended.

## Raw Queries

If we wanted to perform a request where we get the raw json data for some reason, that is also possible within `bavapi`.

However, we will need to use the `Query` object in conjunction with the `raw_query` function to make the query.

In [ ]:
raw_res = bavapi.raw_query(
    TOKEN,
    "companies",
    bavapi.Query(filters=bavapi.filters.FountFilters(name="Meta")),
)
raw_res[0]

## Advanced Usage

>_**NOTE:**_ This section is more geared towards using `bavapi` in other libraries or applications, or when having to make many separate queries. For general Jupyter notebook use, the examples above should work in most cases.

`bavapi` provides a `Client` class that provides a direct `async` interface for developing asynchronous programs or when having to make many separate queries.

`bavapi.Client` is an asynchronous context manager, similar in use to `httpx.AsyncClient` (which powers `bavapi`) or `requests.Session`.

It also brings performance benefits and more. See the [documentation](https://wppbav.github.io/bavapi-sdk-python/latest/usage/advanced/) for more details.

The `Client` can be used directly in Jupyter notebooks, or within asynchronous functions (defined with `async def`).

Use the `async with` context manager syntax, and `await` the desired endpoint method like so:

In [ ]:
async with bavapi.Client(TOKEN) as client:
    fr_bss = await client.brandscape_data(
        country_code="FR",
        year_number=2022,
        audiences=Audiences.ALL_ADULTS,
    )
fr_bss.head()